# Baseline Popular Recommender
As with any ML project, we start with a non-ML approach as our baseline. It's very common in RecSys project to have popular recommender as not only a benchmark but also an actual component (retrieval stage).

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = "001-baseline-popular"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-21 23:55:20.981 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 001-baseline-popular...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "001-baseline-popular",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/001-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")

# Implement

In [5]:
popular_items_df = (
    train_df.groupby(args.item_col, as_index=False)
    .size()
    .assign(
        score=lambda df: df["size"] / df["size"].max(),
        rec_ranking=lambda df: df["score"]
        .rank(method="first", ascending=False)
        .astype(int),
    )
    .sort_values(["rec_ranking"], ascending=[True])
)

top_popular_items_df = popular_items_df.head(args.top_K)

top_popular_items_df

,parent_asin,size,score,rec_ranking
3554,B01N3ASPNV,1514,1.000000,1
2143,B0086VPUHI,1502,0.992074,2
4337,B07YBXFDYN,1391,0.918758,3
2368,B00BGA9WK2,1216,0.803170,4
1775,B004RMK5QG,1095,0.723250,5
...,...,...,...,...
3113,B00ZQB28XK,341,0.225231,96
2789,B00L59D9HG,338,0.223250,97
933,B001EYUQDW,337,0.222589,98
1428,B0030H8I4K,337,0.222589,99


In [6]:
recommendations_df = (
    val_df[[args.user_col]]
    .drop_duplicates()
    .assign(tmp=1)
    .pipe(
        lambda df: pd.merge(
            df, top_popular_items_df.assign(tmp=1), on="tmp", how="left"
        )
    )[[args.user_col, args.item_col, "score", "rec_ranking"]]
)

recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B01N3ASPNV,1.000000,1
1,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B0086VPUHI,0.992074,2
2,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B07YBXFDYN,0.918758,3
3,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B00BGA9WK2,0.803170,4
4,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B004RMK5QG,0.723250,5
...,...,...,...,...
70795,AEA6ITNCVXCJTE56OAOVRIGKGLTQ,B00ZQB28XK,0.225231,96
70796,AEA6ITNCVXCJTE56OAOVRIGKGLTQ,B00L59D9HG,0.223250,97
70797,AEA6ITNCVXCJTE56OAOVRIGKGLTQ,B001EYUQDW,0.222589,98
70798,AEA6ITNCVXCJTE56OAOVRIGKGLTQ,B0030H8I4K,0.222589,99


# Evaluate

## Ranking metrics

In [7]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
548,AFI3SEGKSS7X7CVOC3HS5U5RAIWQ,B077GG9D5D,5.0,1.0
697,AGDFPA6XVENIY7XJPIOXBUHRJQ6A,B09V25XG1G,5.0,1.0
109,AHMJVCKVHJIT2R5NWWV4HG4TDH6A,B07C2XYDW8,5.0,1.0
197,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B0C7BN9G35,5.0,1.0
797,AF6IUPCJM4FDNOQZNCDOGTLM2M4Q,B09T5VN7D1,4.0,1.0
...,...,...,...,...
850,AFB6FYPPCN33UMUU5536IHXNOHCQ,B001E8WQJA,0.0,18.0
1106,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00DBLBMBQ,0.0,18.0
1003,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B00Z9TMBOU,0.0,18.0
52,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0892HRCST,0.0,19.0


In [8]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
79,AE2AZ2MNROPF33U6SS53VI22OXJA,B01N3ASPNV,1.000000,1,0,NaN
23,AE2AZ2MNROPF33U6SS53VI22OXJA,B0086VPUHI,0.992074,2,0,NaN
91,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBXFDYN,0.918758,3,0,NaN
29,AE2AZ2MNROPF33U6SS53VI22OXJA,B00BGA9WK2,0.803170,4,0,NaN
17,AE2AZ2MNROPF33U6SS53VI22OXJA,B004RMK5QG,0.723250,5,0,NaN
...,...,...,...,...,...,...
72369,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B001EYUQDW,0.222589,98,0,NaN
72375,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0030H8I4K,0.222589,99,0,NaN
72408,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00DBLBMBQ,0.222589,100,0,NaN
72376,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B003BLUA64,NaN,101,0,2.0


In [9]:
ranking_report = log_ranking_metrics(args, eval_df)

## Classification metrics

In [11]:
eval_classification_df = pd.merge(
    val_df,
    popular_items_df[[args.item_col, "score"]],
    on=[args.item_col],
    how="left",
    validate="m:1",
).assign(label=lambda df: df[args.rating_col].gt(0).astype(int))
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,score,label
0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B01GY35QPU,0.0,1638057548682,11638,1902,Video Games,Detroit Become Human - PlayStation 4,[Detroit: Become Human is the latest title in ...,"[Video Games, PlayStation 4, Games]",21.4,"[3579, 878, 434, 1579, 3669, 3322, 3495, 2339,...",0.031704,0
1,AEMYS2WV33NDV3OAJFOM7SSSIDTA,B0088TN7BO,0.0,1640914537652,14788,2180,Video Games,LEGO Lord of the Rings - Nintendo 3DS,"[Product Description, Based on The Lord of the...","[Video Games, Legacy Systems, Nintendo Systems...",19.95,"[-1, -1, -1, -1, -1, 3244, 2872, 2621, 3048, 3...",0.118890,0
2,AEXKGQQMYQQUNWVGD66TG3VT4V4A,B00IPTUJ8G,0.0,1653120417877,19789,4619,Video Games,Borderlands 2,"[Borderlands 2 for PS Vita, View Larger, View ...","[Video Games, Legacy Systems, PlayStation Syst...",48.1,"[-1, -1, -1, -1, -1, 2358, 3142, 4521, 2292, 2...",0.027741,0
3,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B00CMQTVK0,0.0,1643392366551,6414,1765,Video Games,Xbox One with Kinect (Day One Edition),"[Get more with Xbox One., Introducing Xbox One...","[Video Games, Xbox One, Consoles]",589.99,"[-1, -1, -1, -1, 3142, 4274, 4345, 809, 416, 1...",0.263540,0
4,AGIJWTPKBANKWNEM2AHK7PWTBYLQ,B01BF9X9VQ,0.0,1647513889874,1195,2412,Video Games,World of Warcraft: Legion - Standard Edition -...,"[Kingdoms will burn, The Burning Legion surges...","[Video Games, PC, Games]",28.6,"[170, 3609, 1588, 1467, 3786, 1627, 4234, 3056...",0.025760,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AF5T2J7T33UFSPUGCKTPTGU7EY7A,B008I2LRMC,0.0,1654892101229,14749,1743,Video Games,F1: 2012,"[Product Description, F1 2012 is designed to b...","[Video Games, Legacy Systems, Xbox Systems, Xb...",33.03,"[601, 1842, 3391, 2640, 761, 3624, 1079, 2656,...",0.028402,0
1894,AGXQLR7TWHVUQLDBKKOBI4OQQGRQ,B01N3ASPNV,0.0,1637285457188,8208,3069,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91,"[-1, -1, 4227, 1039, 1352, 2619, 2655, 1305, 1...",1.000000,0
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,9221,2581,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 2705, 3691, 1804, 111, 3330, 1775, 2979, ...",0.078600,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,4173,3191,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2379, 1799, 1932, 1568, 161]",0.085865,1


In [12]:
eval_classification_df["score"].describe().T

count    1898.000000
mean        0.111546
std         0.154105
min         0.009908
25%         0.027081
50%         0.056143
75%         0.126816
max         1.000000
Name: score, dtype: float64

In [13]:
classification_report = log_classification_metrics(
    args, eval_classification_df, target_col="label", prediction_col="score"
)

## Loss
We can estimate what kinds of MSE loss should be the upper bound model training based on measuring the loss when asking the model to naively predict the mean rating for every item.

In [14]:
naive_prediction = train_df[args.rating_col].mean()

naive_mse = (
    (
        val_df[args.rating_col]
        - val_df.assign(naive_prediction=naive_prediction)["naive_prediction"]
    )
    .apply(lambda s: s * s)
    .mean()
)
logger.info(
    f"Val MSE = {naive_mse:,.2f} given naive_prediction={naive_prediction:,.1f}"
)

2024-09-21 23:55:23.418 | INFO     | __main__:<module>:8 - Val MSE = 5.12 given naive_prediction=2.2


# Clean up

In [15]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/21 23:55:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run 001-baseline-popular at: http://localhost:5003/#/experiments/1/runs/c4156fca060e4bc0a3df4ab6a5c332ab.
2024/09/21 23:55:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
